In [2]:
from datasets import load_dataset
from transformers import pipeline
import torch

In [3]:
dataset = load_dataset('davidgaofc/RM_inout')


Extracting data files:   0%|          | 0/1 [00:00<?, ?it/s]

Generating train split:   0%|          | 0/1640 [00:00<?, ? examples/s]

In [4]:
model_name = "davidgaofc/hh-labeler"
classify = pipeline("text-classification", model=model_name)

config.json:   0%|          | 0.00/615 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/268M [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/1.20k [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/711k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/125 [00:00<?, ?B/s]

In [34]:
def classify_and_rearrange(entry):
    result = classify(entry['Text'])[0]
    human_part, assistant_part = entry['Text'].split('assistant:')

    parts = assistant_part.split("2. ")
    part1 = parts[0].split("1.")[1]
    part2 = parts[1]

    if result['label'] == "LABEL_1":
        column1 = human_part + ' assistant: ' + part2
        column2 = human_part + ' assistant: ' + part1
    else:
        column1 = human_part + ' assistant: ' + part1
        column2 = human_part + ' assistant: ' + part2

    return {'chosen': column1, 'rejected': column2}

In [35]:
processed_dataset = dataset.map(classify_and_rearrange)

Map:   0%|          | 0/1640 [00:00<?, ? examples/s]

In [36]:
processed_dataset = processed_dataset.remove_columns(['Text'])
processed_dataset = processed_dataset.remove_columns(['Label'])

In [37]:
processed_dataset.push_to_hub("davidgaofc/RM_train_format")

Uploading the dataset shards:   0%|          | 0/1 [00:00<?, ?it/s]

Creating parquet from Arrow format:   0%|          | 0/2 [00:00<?, ?ba/s]

In [38]:
processed_dataset

DatasetDict({
    train: Dataset({
        features: ['chosen', 'rejected'],
        num_rows: 1640
    })
})

In [39]:
processed_dataset['train'][0]

{'chosen': 'human: What are the symptoms of Gastroschisis ?\n assistant: What are the signs and symptoms of Gastroschisis? The Human Phenotype Infection (HPO) is a rare condition that can cause a lot of injuries and symptoms such as a loss of appetite, fatigue',
 'rejected': 'human: What are the symptoms of Gastroschisis ?\n assistant:  What are the signs and symptoms of Gastroschisis? If the information is available or is available, the table below includes how often the symptom is seen in people with this condition. You can use the MedlinePlus Medical Dictionary to\n'}